# Algorithm
## Graduation project - Annemijn Portier

----
This file shows how the algorithm is executed with new data from the prototype.

----
## Database connection
First, a connection will be made to the database to retrieve the data.

In [82]:
#!/usr/bin/env python
import pymysql
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import mysql.connector

#connect to database on localhost
mydb = mysql.connector.connect(host='localhost',
                                         database='loans',
                                         user='root',
                                         password='')

The table data is retrieved and placed in a data frame. This data shows the data of customers who have already applied for a loan on Geld.nl

In [83]:
pd.set_option('display.max_columns', None) #show all columns
df = pd.read_sql("select * from data", mydb) #create new data frame
df.head()

,id,date,providerName,reviewScore,interestRate,typeLoan,loanDuration,monthlyRepaymentAmount,loanPurpose,loanAmount,loanCategory,age,typeIncome,netIncome,agePartner,typeIncomePartner,netIncomePartner,children,province,residence,sortedBy,preferenceCompare products,positionProduct,consumptionValues,goals
0,1,2019-03-14 00:00:00,3,77,4,1,120,150,1,17000,0,35,1,3347,34,1,840,1,1,158,1,0,0,0,3
1,2,2019-03-14 00:00:00,3,77,5,1,24,150,6,10000,0,27,2,5000,0,0,0,2,4,296,1,0,0,0,1
2,3,2019-03-14 00:00:00,3,77,3,1,120,150,1,33000,0,51,1,2584,54,1,2366,1,1,1164,1,0,0,0,3
3,4,2019-03-14 00:00:00,3,77,3,1,120,150,6,26000,0,52,1,2300,0,0,0,1,1,52,1,0,0,0,3
4,5,2019-03-14 00:00:00,3,77,4,1,120,150,2,15000,0,27,1,1100,0,0,0,2,2,870,1,0,0,0,3


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8368 entries, 0 to 8367
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   id                          8368 non-null   int64 
 1   date                        8368 non-null   object
 2   providerName                8368 non-null   int64 
 3   reviewScore                 8368 non-null   int64 
 4   interestRate                8368 non-null   int64 
 5   typeLoan                    8368 non-null   int64 
 6   loanDuration                8368 non-null   int64 
 7   monthlyRepaymentAmount      8368 non-null   int64 
 8   loanPurpose                 8368 non-null   int64 
 9   loanAmount                  8368 non-null   int64 
 10  loanCategory                8368 non-null   int64 
 11  age                         8368 non-null   int64 
 12  typeIncome                  8368 non-null   int64 
 13  netIncome                   8368 non-null   int6

----
## Predicting variable review score
The variable review score will be predicted with the new data of the prototype.

In [85]:
y = df['reviewScore']#create the y-variable
X = df[['loanPurpose', 'loanAmount', 'age', 'typeIncome', 'netIncome','agePartner','typeIncomePartner', 
               'netIncomePartner','children','positionProduct','consumptionValues', 'goals']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44) #split the data into test and training set


#Create a Gaussian Classifier
rf=RandomForestClassifier(bootstrap= True, max_depth= None, max_features= 3, min_samples_split= 8, n_estimators= 343)

#Train the model using the training sets y_pred=clf.predict(X_test)
rf=rf.fit(X_train,y_train)

y_pred_rs=rf.predict(X_test)

The table customerdata is retrieved and placed in a data frame. This data shows the new data that has been retrieved from the prototype where new customers want to apply for a loan. The last row is retrieved with **tail(1)** and placed in a new data frame. This row is the last customer that entered their information in the prototype and with this row new predictions will be made to give advice.

In [86]:
df2 = pd.read_sql("select * from customerdata", mydb).tail(1) #only select last row of data frame

df2.head()

,id,loanPurpose,loanAmount,monthamount,monthamount2,monthamount3,loanCategory,age,typeIncome,netIncome,agePartner,typeIncomePartner,netIncomePartner,children,province,consumptionValues,goals,positionProduct,reviewScore,interestRate,typeLoan,loanDuration,monthlyRepaymentAmount,reviewScore2,reviewScore3,interestRate2,interestRate3,loanDuration2,loanDuration3,highestReviewScore,lowestLoanDuration,persuasion,persuasion2,persuasion3
10,39,1,25000,429,2166,729,0,55,1,3000,44,1,3400,2,11,3,7,0,77,3,1,60,0,76,79,4,5,12,36,79,12,1086,170,201


To put all new data with the same customer in the database, the customer's ID has been retrieved.

In [87]:
id_customer = df2['id'].to_string(index=False)
print(id_customer)

 39


With the data frame in which the last customer is placed, a prediction is made for the variable review score.

In [88]:
Xnew = df2[['loanPurpose', 'loanAmount', 'age', 'typeIncome', 'netIncome','agePartner','typeIncomePartner', 
               'netIncomePartner','children','positionProduct','consumptionValues', 'goals']]

ynew = rf.predict(Xnew) #make new prediction

Yrs = ''.join(str(e) for e in ynew)
Yrs = int(Yrs)
print(Yrs)

77


### Predicted probabilties
Predicted probabilities looks at the probability of a class that is calculated from the algorithm. With the predicted probabilities I selected the top 3 highest predicted probabilities for the variable review score.

In [89]:
probs = rf.predict_proba(Xnew) #create predicted probabilties
probs = sorted( zip( rf.classes_, probs[0] ), key=lambda x:x[1] )[-3:] #show the top 3 predicted probabilties
probs

[(79, 0.052145975178045145),
 (76, 0.1099168324386983),
 (77, 0.7620777020193931)]

These predicted probabilities have been added to new variables to be stored to the database later.

In [90]:
Yrs2 = probs[1]
Yrs2 = ' '.join(map(str, Yrs2))
Yrs2 = Yrs2[:2]
Yrs2 = int(Yrs2)
Yrs2

76

In [91]:
Yrs3 = probs[0]
Yrs3 = ' '.join(map(str, Yrs3))
Yrs3 = Yrs3[:2]
Yrs3 = int(Yrs3)
Yrs3

79

### Label highest review score
To create the label highest review score for the prototype, the predicted probabilties that have the highest review score were examined. The highest review score was then stored in a new variable.

In [92]:
highestReviewScore = int(max(Yrs, Yrs2, Yrs3))
highestReviewScore = int(highestReviewScore)
print(highestReviewScore)

79


----
## Predicting variable interest rate
The variable interest rate will be predicted with the new data of the prototype.

In [93]:
y = df['interestRate']#create the y-variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44) #split the data into test and training set


#Create a Gaussian Classifier
rf=RandomForestClassifier(bootstrap= True, max_depth= None, max_features= 3, min_samples_split= 8, n_estimators= 343)

#Train the model using the training sets y_pred=clf.predict(X_test)
rf=rf.fit(X_train,y_train)

y_pred_rs=rf.predict(X_test)

With the data frame in which the last customer is placed, a prediction is made for the variable interest rate.

In [94]:
ynew = rf.predict(Xnew)

Yir = ''.join(str(e) for e in ynew) #make new prediction
Yir = int(Yir)
print(Yir)

3


### Predicted probabilties
With the predicted probabilities I selected the top 3 highest predicted probabilities for the variable interest rate.

In [95]:
probs_Yir = rf.predict_proba(Xnew) #create predicted probabilties
probs_Yir = sorted( zip( rf.classes_, probs_Yir[0] ), key=lambda x:x[1] )[-3:] #show the top 3 predicted probabilties
probs_Yir

[(5, 0.021714429075945108), (4, 0.24187696009853457), (3, 0.7275093306434416)]

These predicted probabilities have been added to new variables to be stored to the database later.

In [96]:
Yir2 = probs_Yir[1]
Yir2 = ' '.join(map(str, Yir2))
Yir2 = Yir2[:2]
Yir2 = int(Yir2)
Yir2

4

In [97]:
Yir3 = probs_Yir[0]
Yir3 = ' '.join(map(str, Yir3))
Yir3 = Yir3[:2]
Yir3 = int(Yir3)
Yir3

5

---
## Predicting variable type of loan
The variable type of loan will be predicted with the new data of the prototype.

In [98]:
y = df['typeLoan']#create the y-variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44) #split the data into test and training set


#Create a Gaussian Classifier
rf=RandomForestClassifier(bootstrap= True, max_depth= None, max_features= 3, min_samples_split= 8, n_estimators= 343)

#Train the model using the training sets y_pred=clf.predict(X_test)
rf=rf.fit(X_train,y_train)

y_pred_rs=rf.predict(X_test)

Based on the data analysis in the notebook *Data cleaning, data analysis and machine learning algorithms.ipynb*, it was noticed that type of loan has very few data records of the class revolving credit. Therefore, it was decided to only predict the class personal loan because this class is only correctly predicted.

In [99]:
ynew = rf.predict(Xnew)

Ytl = ''.join(str(e) for e in ynew) #make new prediction
Ytl = int(Ytl)
print(Ytl)

1


---
## Predicting variable loan duration
The variable loan duration will be predicted with the new data of the prototype.

In [100]:
y = df['loanDuration']#create the y-variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44) #split the data into test and training set


#Create a Gaussian Classifier
rf=RandomForestClassifier(bootstrap= True, max_depth= None, max_features= 3, min_samples_split= 8, n_estimators= 343)

#Train the model using the training sets y_pred=clf.predict(X_test)
rf=rf.fit(X_train,y_train)

y_pred_rs=rf.predict(X_test)

In [101]:
ynew = rf.predict(Xnew)

Yld = ''.join(str(e) for e in ynew) #make new prediction
Yld = int(Yld)
print(Yld)

60


### Predicted probabilties
With the predicted probabilities I selected the top 3 highest predicted probabilities for the variable loan duration.

In [102]:
probs_Yld = rf.predict_proba(Xnew) #create predicted probabilties
probs_Yld = sorted( zip( rf.classes_, probs_Yld[0] ), key=lambda x:x[1] )[-3:] #show the top 3 predicted probabilties
probs_Yld

[(36, 0.08779622547844124),
 (120, 0.3456258877177244),
 (60, 0.42332577124851173)]

These predicted probabilities have been added to new variables to be stored to the database later.

In [103]:
Yld2 = probs_Yld[1]
Yld2 = ' '.join(map(str, Yld2))
Yld2 = Yld2[:2]
Yld2 = int(Yld2)
Yld2

12

In [104]:
Yld3 = probs_Yld[0]
Yld3 = ' '.join(map(str, Yld3))
Yld3 = Yld3[:2]
Yld3 = int(Yld3)
Yld3

36

### Label lowest loan duration
To create the label lowest loan duration for the prototype, the predicted probabilties that have the lowest loan duration were examined. The lowest loan duration was then stored in a new variable.

In [105]:
lowestLoanDuration = int(min(Yld, Yld2, Yld3))
lowestLoanDuration = int(lowestLoanDuration)
print(lowestLoanDuration)

12


---
## Personalized persuasive information
Based on the persuasion knowledge theory in the academic paper, it was important to create informative and subtle personalized persuasive information that leads to an improved decision quality. With each predicted advice, the personalized persuasive information shows how many customers with the same loan purpose have also applied for the same provider. 

In [106]:
loanPurpose = df2.iloc[0]['loanPurpose'] #show which loan purpose the new customer has
loanPurpose

1

To find out how many customers with the same loan purpose per predicted advice chose the same provider, I calculated the number of rows that has the same loan purpose and same provider.

In [107]:
df_subset = df.loc[df['loanPurpose'] == loanPurpose] #create new data frame with rows that has the same loan purpose as the new customer
df_subset.head()

,id,date,providerName,reviewScore,interestRate,typeLoan,loanDuration,monthlyRepaymentAmount,loanPurpose,loanAmount,loanCategory,age,typeIncome,netIncome,agePartner,typeIncomePartner,netIncomePartner,children,province,residence,sortedBy,preferenceCompare products,positionProduct,consumptionValues,goals
0,1,2019-03-14 00:00:00,3,77,4,1,120,150,1,17000,0,35,1,3347,34,1,840,1,1,158,1,0,0,0,3
2,3,2019-03-14 00:00:00,3,77,3,1,120,150,1,33000,0,51,1,2584,54,1,2366,1,1,1164,1,0,0,0,3
8,9,2019-03-16 00:00:00,3,77,4,1,108,150,1,15000,0,57,1,2750,55,1,880,2,3,1170,1,0,0,0,3
12,13,2019-03-16 00:00:00,3,77,3,1,120,150,1,25000,0,26,1,2000,0,0,0,2,2,3,1,0,0,0,3
19,20,2019-03-17 00:00:00,3,77,4,1,60,150,1,17000,0,54,1,3600,58,1,1500,2,6,725,1,1,0,1,2


In [108]:
amountrows = df_subset.shape[0] #calculate the amount of rows with same loan purpose
amountrows

2415

In [109]:
df_subset2 = df_subset.loc[df_subset['reviewScore'] == Yrs] #in the new data frame with the selected loan purpose, only select the rows with the same review score
df_subset2.head()

,id,date,providerName,reviewScore,interestRate,typeLoan,loanDuration,monthlyRepaymentAmount,loanPurpose,loanAmount,loanCategory,age,typeIncome,netIncome,agePartner,typeIncomePartner,netIncomePartner,children,province,residence,sortedBy,preferenceCompare products,positionProduct,consumptionValues,goals
0,1,2019-03-14 00:00:00,3,77,4,1,120,150,1,17000,0,35,1,3347,34,1,840,1,1,158,1,0,0,0,3
2,3,2019-03-14 00:00:00,3,77,3,1,120,150,1,33000,0,51,1,2584,54,1,2366,1,1,1164,1,0,0,0,3
8,9,2019-03-16 00:00:00,3,77,4,1,108,150,1,15000,0,57,1,2750,55,1,880,2,3,1170,1,0,0,0,3
12,13,2019-03-16 00:00:00,3,77,3,1,120,150,1,25000,0,26,1,2000,0,0,0,2,2,3,1,0,0,0,3
19,20,2019-03-17 00:00:00,3,77,4,1,60,150,1,17000,0,54,1,3600,58,1,1500,2,6,725,1,1,0,1,2


In [110]:
amountrows2 = df_subset2.shape[0] #calculate the amount of rows with same loan purpose and same review score
amountrows2

1086

This is also done with the other two loans that are predicted for the customer.

In [111]:
df_subset3 = df_subset.loc[df_subset['reviewScore'] == Yrs2] #in the new data frame with the selected loan purpose, only select the rows with the same review score of second loan
df_subset3.head()

,id,date,providerName,reviewScore,interestRate,typeLoan,loanDuration,monthlyRepaymentAmount,loanPurpose,loanAmount,loanCategory,age,typeIncome,netIncome,agePartner,typeIncomePartner,netIncomePartner,children,province,residence,sortedBy,preferenceCompare products,positionProduct,consumptionValues,goals
1686,1687,2019-08-12 00:00:00,0,76,5,1,60,150,1,25000,0,28,1,2200,0,0,0,2,9,67,2,0,9,2,3
1781,1782,2019-08-14 00:00:00,0,76,3,1,60,150,1,25000,0,32,1,2803,29,2,1000,1,7,242,2,0,1,2,3
1784,1785,2019-08-15 00:00:00,0,76,3,1,72,150,1,35000,0,31,1,2500,0,0,0,2,2,870,1,0,1,2,3
1825,1826,2019-08-16 00:00:00,0,76,5,1,60,150,1,10000,0,46,1,1910,0,0,0,1,2,892,1,0,5,0,3
1827,1828,2019-08-16 00:00:00,0,76,3,1,72,150,1,24000,0,48,1,3500,49,1,1900,1,9,404,1,0,1,2,3


In [112]:
amountrows3 = df_subset3.shape[0] #calculate the amount of rows with same loan purpose and same review score of second loan
amountrows3

170

In [113]:
df_subset4 = df_subset.loc[df_subset['reviewScore'] == Yrs3] #in the new data frame with the selected loan purpose, only select the rows with the same review score of third loan
df_subset4.head()

,id,date,providerName,reviewScore,interestRate,typeLoan,loanDuration,monthlyRepaymentAmount,loanPurpose,loanAmount,loanCategory,age,typeIncome,netIncome,agePartner,typeIncomePartner,netIncomePartner,children,province,residence,sortedBy,preferenceCompare products,positionProduct,consumptionValues,goals
1405,1406,2019-08-01 00:00:00,1,79,4,1,60,150,1,17500,0,28,1,2500,0,0,0,2,4,296,2,0,2,2,3
1460,1461,2019-08-03 00:00:00,7,79,6,2,60,100,1,5000,0,35,1,2150,36,7,0,1,7,380,1,0,2,0,3
1537,1538,2019-08-06 00:00:00,1,79,4,1,120,150,1,15000,0,33,1,2380,32,6,1000,1,10,1118,1,0,2,0,3
1590,1591,2019-08-08 00:00:00,1,79,6,2,60,150,1,10000,0,49,1,1500,62,2,1500,1,4,296,1,0,3,0,3
1684,1685,2019-08-12 00:00:00,7,79,6,2,60,200,1,5000,0,31,5,1860,0,0,0,2,11,1024,1,0,2,0,3


In [114]:
amountrows4 = df_subset4.shape[0] #calculate the amount of rows with same loan purpose and same review score of third loan
amountrows4

201

---
## Monthly amount
With the new predicted advices, I calculated the monthly amount that the customer must pay.

First I collected the total amount that the new customer has to pay.

In [115]:
loanAmount = df2.iloc[0]['loanAmount']
loanAmount

25000

Then I calculated the monthly amount for every predicted loan with the following formula:

In [116]:
monthamount = int(((loanAmount*Yir/100)+loanAmount)/Yld)
monthamount

429

In [117]:
monthamount2 = int(((loanAmount*Yir2/100)+loanAmount)/Yld2)
monthamount2

2166

In [118]:
monthamount3 = int(((loanAmount*Yir3/100)+loanAmount)/Yld3)
monthamount3

729

---
## Store new data of customer in database
By predicting all new data for the new customer, it can be stored in the database with table customerdata. With this data the results overview can be shown in the prototype.

In [119]:
mycursor = mydb.cursor()
#update new data of the customer
mycursor.execute("update customerdata set reviewScore = %s, reviewScore2 = %s, reviewScore3 = %s, interestRate = %s, interestRate2 = %s, interestRate3 = %s, typeLoan = %s, loanDuration = %s, loanDuration2 = %s, loanDuration3 = %s, highestReviewScore = %s, lowestLoanDuration = %s, persuasion = %s, persuasion2 = %s, persuasion3 = %s, monthamount = %s, monthamount2 = %s, monthamount3 = %s where id = %s;", (Yrs, Yrs2, Yrs3, Yir, Yir2, Yir3, Ytl, Yld, Yld2, Yld3, highestReviewScore, lowestLoanDuration, amountrows2, amountrows3, amountrows4, monthamount, monthamount2, monthamount3, id_customer))

To use this algorithm in the prototype it must be converted into a python file.

In [120]:
!jupyter nbconvert --to script algorithm.ipynb

[NbConvertApp] Converting notebook algorithm.ipynb to script
[NbConvertApp] Writing 12301 bytes to algorithm.py
